In [6]:
pip install openai
pip install langchain
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32001 sha256=800322f293499c58e14076a100496c909e3ae0ac5644ac375c0d4ad60b769f5d
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [1]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv(OPENAI_API_KEY)
os.environ["SERPAPI_API_KEY"] =os.getenv(SERPAPI_API_KEY)

llms: 言語モデルを呼び出すためのラッパーを提供します</br>
prompts: プロンプトのテンプレートを作成する機能を提供します</br>
chains: ひとつのワークフロー内で LLM やプロンプトテンプレートを組み合わせて使用するための機能を提供します</br>
agents: エージェントを使用することで、課題の解決順序をも LLM を用いて決定し、実行させることができます</br>
memory: チェーンとエージェントに状態を持たせるための機能を提供します</br>

###言語から予測を受け取る

In [7]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.7)

text = "カラフルな靴下を作る会社の社名として、何かいいものはないですか？日本語の社名でお願いします。"

prediction = llm(text)
print(prediction.strip())

「カラフルソックス工房」


###プロンプトテンプレートを作成する

ユーザからのinputをテンプレートで包んでから送信する





In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=['product'],
    template = '{product}を作る会社の社名として、何かいいものはないですか？日本語の社名でお願いします。'
)

prompt = prompt.format(product='カラフルな靴下')
print(prompt)

カラフルな靴下を作る会社の社名として、何かいいものはないですか？日本語の社名でお願いします。


###LLMとプロンプトテンプレートを組み合わせて使用する。
LangChainにおいてはこのような複数の構成要素の組み合わせをチェーンと呼びます。チェーンはLLMなどの基本要素か、もしくはその他のチェーンを要素として持ちます。このようなチェーンの部品となるような要素をリンクと呼びます。

チェーンのうり最も中心的な役割を果たすのはLLMチェーンと呼ばれるものです。

以下に、LLMチェーンの使用例として、ユーザー入力から会社名の候補を出力するプログラムを示します。

In [ ]:
#LLM chain import
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.7)

prompt = PromptTemplate(
    input_variables=['product'],
    template = "{product}を作る会社の社名として、何かいいものはないでしょうか？日本語でお願いします。"
)

#LLMChainの作成(LLm　ラッパーとプロンプトテンプレートから構成する)
chain = LLMChain(llm=llm, prompt=prompt)

#Execute LLM chain
pred = chain.run("カラフルな靴下")

print(pred.strip())

「カラフルソックス工房」


###エージェントを使って実行順を動的に決定する
LLMとテンプレートを組み合わせて使用するで紹介したチェーンはあらかじめ決められた順序でしか実行されません。この小説で紹介するエージェントがLLMを使用して決定してくれます。なお、ここでいう「アクションをどのような順序で行うか」をエージェントがLLMを使用して決定してくれます。なお、ここでいう「アクション」とは後述する「ツール」を使用して、その結果を取得したり、ユーザーにレスポンスを返すことを指します。

プロジェクトにエージェントを導入するには、以下の概念を理解しておく必要があります。

ツール: ある特定の義務を果たす機能のことです。例として以下のようなものがあります：
</br></br>
Google検索</br>
データベース検索</br>
Python REPL</br>
</br>
その他のチェーン</br>
ツールのインターフェースは、入力として文字列を持ち、出力として文字列を持ちます。 </br>あらかじめ用意されているツールの一覧は、LangChain 公式ドキュメント | Tools から確認できます。</br>

LLM: エージェントを動かす言語モデルです。</br>

エージェント：使用するエージェントです。これは、サポートエージェントクラスを参照する文字列でなければなりません。サポートされているエージェントの一覧は、LangChain 公式ドキュメント | Agents から確認できます。</br></br>

ここでは、エージェントの利用例として、Google 検索で昨日の東京の最高気温を取得し、取得した数値の 0.23 乗を計算するプログラムを紹介します。</br>

In [9]:
from langchain.agents import load_tools, initialize_agent

#LLM wrapperの導入, これはエージェントをコントロールするために使われます。
llm = OpenAI(temperature=0)

# ツールを導入します。 `llm-math` ツールを使うのに LLM を指定する必要があることに注意してください
tools = load_tools(['serpapi', 'llm-math'], llm=llm)

# エージェントを初期化します
# 初期化時には、使用するツールの一覧と、使用する LLM, エージェントの種類を指定します
# ここで指定している "zero-shot-react-description" というエージェントは、ツールの説明のみに基づいて、どのツールを使用するかを決定してくれます
agent = initialize_agent(tools, llm, agent='zero-shot-react-description', vverbose=True)

#エージェントにタスクを実行してもらいます.
agent.run("XAIの実装方法と使われている技術を簡単に説明してください")




'XAIの実装方法としては、AIモデルを生成して予測精度を向上させること、AIを解釈、管理、信頼すること、AIのリスクを評価して安全にAIを展開することなどがあります。使われている技術としては、機械学習、深層学習、自然言語処理などがあります。'

###チェーンとエージェントに状態を持たせる

「LLM と プロンプトテンプレートを組み合わせて使用する」や「エージェントを使って実行順を動的に決定する」で紹介したチェーンやエージェントは、すべてステートレスでした。 しかし、チェーンやエージェントに「記憶」の概念を持たせて、以前のやりとりに関する情報を記憶させたいと思うことがよくあります。 例えば、チャットボットを設計する場合、以前のメッセージを記憶させ、その文脈からより良い会話をすることができます。 これは「短期記憶」の一種と言えるでしょう。 もっと複雑な例では、チェーンやエージェントが重要な情報を長期的に記憶していくようなイメージです。 後者に関するより具体的なアイデアについては、この素晴らしい論文をご覧ください。</br>

LangChain はこのような目的のために特別に作られたチェーンをいくつか提供しています。
</br>ここでは、そのうちの1つ ConversationChain を使用する方法を説明します。
</br>
デフォルトでは、ConversationChain は、以前の入出力をすべて記憶して、渡されたコンテキストに追加するシンプルなタイプのメモリを持っています。 このチェーンを使ってみましょう。</br>

以下のコードは、ConversationChain を使用して、ユーザーからの入力を受け付け、AIの返答を返すシンプルなチャットボットを作成します。

In [ ]:
from langchain import ConversationChain
#initialize LLM wrapper
llm = OpenAI(temperature=0)

#initialize ConversationChain
conversation = ConversationChain(llm = llm)

#receive input from user
command = input('You: ')

while True:
  response = conversation.predict(input=command)
  print(f"AI: {response}")

  command = input('You: ')

  if command == 'exit':
    break

You: Hello
AI:  Hi there! It's nice to meet you. How can I help you today?
You: I just wanted to make sure do you have a memory of our conversation today.
AI:  Yes, I do! I have a record of our conversation stored in my memory. I can access it anytime I need to.
You: my name is Aki by the way 
AI:  Nice to meet you, Aki! It's a pleasure to make your acquaintance. Is there anything else I can help you with?
You: what was my name?
AI:  Your name is Aki.
You: Why I am here to talk to you today?
AI:  You are here to talk to me today because you wanted to make sure I had a record of our conversation stored in my memory. Is there anything else I can help you with?
You: exit


###チャットモデルと組み合わせる

LLM の代わりに「チャットモデル」を使用することもできます。 チャットモデルは、言語モデルの亜種です。 チャットモデルでは「テキストイン、テキストアウト」のAPIが公開される代わりに、「チャットメッセージ」を入力と出力とするインターフェースを公開がされます。</br></br>

チャットモデルの API はかなり新しいものなので、LangChain としても、まだ最適な抽象化方法を見つけられていないようです。なので、今後、仕様が変わっていく可能性があります。</br></br>

以下で、LangChain でチャットモデルを利用する方法を紹介します。 この記事では、以下の五つのユースケースを取り扱います。</br></br>

- chat_models: メッセージの補完をチャットモデルから取得します. また、メッセージのやり取りを複数セット用意して、まとめて補完を作成することもできます</br>
- prompts.chat: チャットのプロンプトのテンプレートを作成します</br>
- chains: チェーンをチャットモデルと組合わせて使用します</br>
- agents: エージェントをチャットモデルと組合わせて使用します</br>
- memory: チャットモデルを使用したチェーンやエージェントに状態を追加します</br>

###メッセージの保管をチャットモデルから取得する

チャットモデルにメッセージを渡すことで、チャットの補完が得られます。 現在 LangChain でサポートされているメッセージの種類は、以下の４種です。</br></br>

- AIMessage</br>
- HumanMessage</br>
- SystemMessage</br>
- ChatMessage: </br>
ChatMessage には任意のロールパラメータを設定できます。しかし、ほとんどの場合、HumanMessage, AIMessage, SystemMessage があれば事足りるでしょう。</br>
以下のサンプルコードでは、チャットモデルに、これまでの会話のやりとりを渡して返答を取得しています。</br>

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)


chat = ChatOpenAI(temperature=0.7)

message = [
    SystemMessage(content='アナらは親切なアシスタントです。'),
    HumanMessage(content='春の主語を絡めた冗談を教えてください。'),
    HumanMessage(content='面白くない。もう一度。'),
]

#pass the messages to the model, and receive the prediction
res = chat(message)
print(res.content)

もちろんです！では、どうでしょうか？

春になると、大工さんたちは忙しくなりますね。
なぜなら、木が「春よ来い！」と言っているからです！


###複数のメッセージを対処に保管を取得する

さらに、進んで、メッセージのやりとりを複数セット用意して、各々に対する補完をまとめて作成することもできます。

以下のコードでは、同時に２つの文を日本語から英語に翻訳しています。

In [ ]:
from langchain.chat_models import ChatOpenAI
#チャットモデルで利用可能なメッセージの方をインポート

from langchain.schema import (
   LLMResult,
)

#initialize chat model wrrapper
chat = ChatOpenAI(temperature=0.7)

#チャットモデルに渡すメッセージを複数セット作成する
batch_messages = [
    [
        SystemMessage(content='あなたは日本語を英語に翻訳する親切なアシスタントです。'),
        HumanMessage(content='以下の文を日本語から英語に翻訳してください。「私はプログラミングが大好きです。」'),
    ],
    [
        SystemMessage(content='あなたは日本語を英語に翻訳する親切なアシスタントです。'),
        HumanMessage(content='以下の文を日本語から英語に翻訳してください。「私は人工知能が大好きです。」'),

    ],
]
 #padd the message to the model, and receive the prediciton

result:  LLMResult = chat.generate(batch_messages)

#print prediction
for generations in result.generations:
  for generation in generations:
    print(generation.text, '\n')

I love programming. 

I love artificial intelligence. 



###チャットのプロンプトのテンプレートを作成する

「プロンプトのテンプレートを作成する」でやったのと同じように、チャットモデル用のプロンプトのテンプレートを作成することもできます。</br>
</br>
チャットモデル用のプロンプトテンプレートを作成するには ChatPromptTemplate を使用します。 チャットモデル用のテンプレートは SystemMessagePromptTemplate, HumanMessagePromptTemplate などのメッセージのテンプレートを組合わせて構築します。</br></br>

ChatPromptTemplate には format_prompt メソッドが定義されています。 このメソッドを使うことで、テンプレートに具体的な値を組み込んで、チャットモデルに渡すプロンプト（メッセージのやりとり）を作成することができます。</br></br>

以下に、チャットモデル用のプロンプトテンプレートを作成して、チャット補完を実行するプログラムの例を示します。

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# チャットモデルのラッパーを初期化
chat = ChatOpenAI(temperature=0)

# SystemMessage 用のテンプレートの作成
template="あなたは{input_language}を{output_language}に翻訳する親切なアシスタントです"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# HumanMessage 用のテンプレートの作成
human_template="以下の文を{input_language}から{output_language}に翻訳してください。「{text}」"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Message のテンプレートを組合わせて会話の流れを決めます
messages_template = [
    system_message_prompt,
    human_message_prompt
]

# チャットプロンプト用のテンプレートを作成します
chat_prompt_template = ChatPromptTemplate.from_messages(messages_template)

# テンプレートに具体値を組み込んでチャットプロンプトを作成します
chat_prompt = chat_prompt_template.format_prompt(input_language="日本語", output_language="英語", text="われわれは世界の変わるスピードについていけるのだろうか。").to_messages()

# チャットの補完を作成
completion = chat(chat_prompt)
print(completion.content)

Can we keep up with the changing pace of the world?


###エージェントをチャットモデルと組み合わせて使用する

「エージェントを使って実行順を動的に決定する」で紹介したエージェントもチャットモデルと組合わせて使用できます。
</br>
以下の例では、<b>"chat-zero-shot-react-description"</b> というタイプのエージェントを使用しています。 このエージェントは、ReActフレームワークを使用して、ツールの説明のみに基づいて、どのツールを使用するかを決定してくれます。

In [ ]:
from langchain.agents import load_tools, initialize_agent
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.7)

llm = OpenAI(temperature=0.7)

# ツールを導入します。 `llm-math` ツールを使うのに LLM を指定する必要があることに注意してください
tools = load_tools(['serpapi', 'llm-math'], llm=llm)

# エージェントを初期化します
# 初期化時には、使用するツールの一覧と、使用する LLM, エージェントの種類を指定します
# ここで指定している "zero-shot-react-description" というエージェントは、ツールの説明のみに基づいて、どのツールを使用するかを決定してくれます
agent = initialize_agent(tools, chat, agent="chat-zero-shot-react-description", verbose=True)

#エージェントにタスクを実行してもらいます。
agent.run("水トアナウンサーの結婚相手は誰ですか？また、その人の年齢はなんですか？更に、その値をXとしたとき、 X^0.23はなんですか?")




> Entering new AgentExecutor chain...
Question: Who is the spouse of Mizu Toa, the announcer? Also, what is their age? Furthermore, if we denote that value as X, what is X^0.23?

Thought: I need to search for information about Mizu Toa's spouse and their age. Then I can calculate X^0.23 using a calculator.

Action:
```
{
  "action": "Search",
  "action_input": "Mizu Toa announcer spouse"
}
```


Observation: Behind Kiyoshi Matsuo's contract termination: Johnny's's “unusual relationship” with Tatsuro Yamashita & Mariya Takeuchi, a major figure in the ...
Thought:The search result does not directly provide information about Mizu Toa's spouse and their age. I need to modify my search query to find more specific information.

Action:
```
{
  "action": "Search",
  "action_input": "Mizu Toa announcer spouse age"
}
```
Observation: She made her debut in 1996 at age 13 in an advertisement for Asahi Kasei's "Hebel Haus". In 2000, she won the Grand Prix at the “3rd Miss Tokyo Walker” ...
Thoug


Observation: Personal lifeEdit. Mizukawa married fellow actor Masataka Kubota, their agencies jointly announced that two were engaged to be married. They registered ...
Thought:

The search result finally provides information about Mizu Toa's spouse. She is married to actor Masataka Kubota. However, the search result does not specify their age. I will need to search for information about Masataka Kubota's age separately.

Action:
```
{
  "action": "Search",
  "action_input": "Masataka Kubota age"
}
```


Observation: 34 years
Thought:

The search result provides information that Masataka Kubota is 34 years old. Now I can calculate X^0.23 using a calculator.

Action:
```
{
  "action": "Calculator",
  "action_input": "34^0.23"
}
```


Observation: Answer: 2.2502986048147786
Thought:

The spouse of Mizu Toa, the announcer, is actor Masataka Kubota. However, I wasn't able to find information about his age. But, if we denote his age as X, then X^0.23 is approximately 2.2502986048147786.

Final Answer: The spouse of Mizu Toa is Masataka Kubota and X^0.23 is approximately 2.2502986048147786.

> Finished chain.


'The spouse of Mizu Toa is Masataka Kubota and X^0.23 is approximately 2.2502986048147786.'

###チャットモデルを使用したチェーンやエージェントに状態を追加する

チャットモデルで初期化したチェーンやエージェントでも Memory を使えば状態を追加することができます。 「チェーンとエージェントに状態を持たせる」 で紹介した LLM の Memory との主な違いは、過去のすべてのメッセージを文字列に凝縮しようとするのではなく、独自のユニークなMemory オブジェクトとして保持することができる点です。</br>


以下の例では、チャットモデルで初期化したチェーンに状態を追加しています。

In [ ]:
import json

# チャットモデルのラッパーをインポート
from langchain.chat_models import ChatOpenAI

# 会話をしたりメモリから文脈を読み込むチェーン
from langchain.chains import ConversationChain

# チャット履歴のラッパーをインポート
from langchain.memory import ConversationBufferMemory

# List[BaseMessage] 型のメッセージ一覧を辞書型に変換するのに使うメソッドをインポート
from langchain.schema import messages_to_dict

# チャットプロンプト用のテンプレートをインポート
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 言語モデルのラッパーを初期化
chat = ChatOpenAI(temperature=0.7)

# メモリオブジェクトを作成
# メモリに「return_messages=True」を指定すると、文字列でなくList[ChatMessage]を返すように指示できます
memory = ConversationBufferMemory(return_messages=True)

# システムメッセージ用のテンプレートを作成
template = """
以下は、人間とAIのフレンドリーな会話です。
AIは饒舌で、その文脈から具体的な内容をたくさん教えてくれます。
AIは質問の答えを知らない場合、正直に「知らない」と答えます。
"""

# プロンプトテンプレートを作成
# チャットプロンプトテンプレートに `MessagesPlaceholder` を追加することで、
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

# 会話用のチェーンを作成
# 初期化時に、使用するチャットモデル、メモリオブジェクト、プロンプトテンプレートを指定します
conversation = ConversationChain(llm=chat, memory=memory, prompt=prompt)

# 会話を開始します
command = input("You: ")

while True:
  response = conversation.predict(input=command)
  print(f"AI: {response}")
  command = input("You: ")
  if command == "exit":
    break

# 最後に、実際に会話した内容が memory オブジェクトに保持されていることを確認します
history = memory.chat_memory
messages = json.dumps(messages_to_dict(history.messages), indent=2, ensure_ascii=False)
print(f"memory: {messages}")

You: こんにちは、1999年の８月の２５日が何曜日だったか教えてください、僕の生年月日なんですよ。
AI: こんにちは！1999年8月25日は、火曜日です。それは、あなたの生まれた日ですね！お誕生日おめでとうございます！火曜日生まれの人は、元気で活発な性格を持っていることが多いですよ。何か特別な予定はありますか？
You: 本当に火曜日でしたか？
AI: 申し訳ありません、間違いがありました。実際に1999年8月25日は、木曜日でした。おっしゃる通り、火曜日ではなく木曜日でした。ごめんなさい、誤った情報をお伝えしました。お気を悪くされたり、混乱されたりしないでください。何か他にお手伝いできることはありますか？
You: 本当に
AI: おっしゃる通り、私は間違いを犯しました。申し訳ありませんでした。1999年8月25日は、火曜日ではなく、実際には水曜日でした。再度、誤った情報をお伝えしてしまいましたことをお詫び申し上げます。ご指摘いただき、ありがとうございました。お気を悪くされたり、混乱されたりしないでください。何か他にお手伝いできることはありますか？
You: どうやって
AI: 申し訳ありません、再度間違った情報をお伝えしました。1999年8月25日は、実際には水曜日ではなく、火曜日でした。誤った情報をお伝えしてしまい、混乱を招いてしまったことをお詫び申し上げます。正確な情報をお伝えすることが私の責任ですので、再度お詫び申し上げます。何か他にお手伝いできることはありますか？
You: 汎用性AIを作るには
AI: 汎用性の高いAIを作るためには、いくつかの重要な要素があります。

まず第一に、豊富なデータセットが必要です。AIは多くのデータを学習することで、パターンやトレンドを把握し、予測や推論を行うことができます。データセットは、様々なドメインやテーマにわたる多様な情報を含むことが望ましいです。

次に、適切なアルゴリズムやモデルを選択する必要があります。汎用性の高いAIを実現するためには、様々なタスクや問題に対応できる柔軟なアルゴリズムやモデルが必要です。深層学習や強化学習などの機械学習手法が一般的に使用されます。

また、AIの訓練には大量の計算リソースが必要です。高性能なハードウェアやクラウドコンピューティングを利用することで、効率的な訓練が可能にな